In [1]:
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.sql import functions as F
import numpy as np
from pyspark.ml.linalg import Vectors

from pyspark.sql import functions as F
import plotly.express as px
from pyspark.ml.clustering import KMeans
from pyspark.sql.types import *

In [2]:
model = Word2VecModel.load('/common/users/shared/cs543_fall22_group3/models/word2vec')
vector_table = model.getVectors()
processed_df = spark.read.json('/common/users/shared/cs543_fall22_group3/combined/combined_processed')

/koko/system/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
processed_df = processed_df.withColumn("cleaned_text",F.split(F.col("cleaned_text"), ","))

In [4]:
processed_df.show()

+--------------------+--------------------+
|        cleaned_text|       selected_text|
+--------------------+--------------------+
|[london, financia...|London's financia...|
|[the, department,...|The Department of...|
|[mayor, gov, plan...|MAYOR: GOV’S PLAN...|
|[girardi, yankees...|Girardi on Yankee...|
|[the, young, turk...|The Young Turks H...|
|[emerge, market, ...|Emerging markets ...|
|[fool, try, devel...|A Fool tries to d...|
|[the, comedy, wri...|The comedy, writt...|
|[another, call, r...|Another Call to R...|
|[blake, shelton, ...|Blake Shelton & G...|
|[read, nico, bran...|Read Nico Branham...|
|[plus, xero, boss...|PLUS: Xero boss r...|
|[awka-the, claim,...|AWKA-THE claim by...|
|[though, subways,...|Though subways an...|
|[forget, not, chr...|‘Forgotten’? Not ...|
|[leader, oppositi...|Leader of the opp...|
|[natalia, kill, s...|Natalia Kills say...|
|[superior, fall, ...|Superior falls to...|
|[thom, patterson,...|By Thom Patterson...|
|[finance, ministe...|EU finance

In [5]:
result = model.transform(processed_df.drop('selected_text'))

In [6]:
result.show()

+--------------------+--------------------+
|        cleaned_text|      output_vectors|
+--------------------+--------------------+
|[london, financia...|[-0.5109536331146...|
|[the, department,...|[-0.3147776111887...|
|[mayor, gov, plan...|[-5.5029392242431...|
|[girardi, yankees...|[0.12310679753621...|
|[the, young, turk...|[-0.2074893119434...|
|[emerge, market, ...|[-0.4438817477361...|
|[fool, try, devel...|[-0.2903663915349...|
|[the, comedy, wri...|[0.14530817537822...|
|[another, call, r...|[-0.0227802122632...|
|[blake, shelton, ...|[0.2999736661076895]|
|[read, nico, bran...|[0.0738221023763929]|
|[plus, xero, boss...|[-0.2142722297992...|
|[awka-the, claim,...|[-0.0676146328735...|
|[though, subways,...|[-0.0927275374531...|
|[forget, not, chr...|[0.20289693609811...|
|[leader, oppositi...|[-0.0717920884490...|
|[natalia, kill, s...|[0.05694128102105...|
|[superior, fall, ...|[-0.0910834123690...|
|[thom, patterson,...|[-0.2146621572605...|
|[finance, ministe...|[-0.493275

In [7]:
result = result.withColumn('features', F.col("output_vectors"))

In [62]:
kmeans = KMeans(k=5)
kmeans.setSeed(1)
kmeans.setMaxIter(10)
k_means_model = kmeans.fit(result)

In [ ]:
k_means_model.write().overwrite().save('/common/users/shared/cs543_fall22_group3/models/k_means')

In [10]:
k_means_model.clusterCenters()

[array([0.06283892]),
 array([-0.31541211]),
 array([-0.12216592]),
 array([-0.57509271]),
 array([0.27676002])]

In [63]:
k_means_model.setPredictionCol("newPrediction")

KMeansModel: uid=KMeans_8db78f979b9c, k=5, distanceMeasure=euclidean, numFeatures=1

In [64]:
transformed = k_means_model.transform(result)

In [65]:
view = transformed.take(10000)

In [ ]:
# 0 -> pop culture
# 1 -> finance
# 2 -> politics
# 3 -> breaking news
# 4 -> sports/entertainment

In [ ]:
for i in view:
    if i[3] == 3:
        print(i[0])
        print(i[3])
        print()

In [ ]:
# 0 -> 
# 1 -> technology?
# 2 -> politics
# 3 -> finance
# 4 -> sports
# 8 -> health

In [28]:
result.write.mode("Overwrite").json('/common/users/shared/cs543_fall22_group3/combined/combined_vectors_result')